In [1]:
%%capture
import re # to handle regular expression
import demoji  # for emojis handle
import random # to generate random number
import inflect  # to handle number to words
import numpy as np # scitific calculation 
import pandas as pd # data manipulation 
from bs4 import BeautifulSoup # handle html tag
import matplotlib.pyplot as plt # plot any display

import torch # deep learning handle
import torchtext # nlp handle
import torch.nn as nn # NN handle
import torch.optim as optim # optimizer handle
import torch.nn.functional as F # all type to DL funciton
from torch.nn.functional import one_hot # encoder-decoder
from torchtext.data.utils import get_tokenizer # tokenize 
print('Successfully import all the libraries')

In [2]:
# Reading the text 
file_path = 'human_chat.txt'
with open(file_path,"r") as f:
    lines = f.readlines()

In [3]:
type(lines[1])

str

In [4]:
soup = BeautifulSoup(str(lines), 'html.parser')
text = soup.get_text()

In [5]:
def preprocess_text(text):
        # Remove HTML tags
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text()

        p = inflect.engine() #101
        # demoji.download_codes()
        # Remove emojis
        text = demoji.replace(text, "")

        # Remove mentions of "Human 1" and "Human 2"
        text = re.sub(r'\b(?:Human 1|Human 2)\b:?', " ", text)

        # Replace numbers with words
        text = re.sub(r'\b\d+\b', lambda x: p.number_to_words(x.group()), text)

        # Remove special characters, keeping only alphabetic and spaces
        text = re.sub('[^a-zA-Z\s]', ' ', text)

        # Replace specific unicode spaces with standard spaces and trim
        text = text.replace(u'\xa0', u' ').replace('\u200a', ' ').strip()

        return text

In [6]:
preprocessed_lines = [preprocess_text(line) for line in lines]

/tmp/ipykernel_11338/3061531194.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [7]:
preprocessed_lines[0:5]

['Hi',
 'What is your favorite holiday',
 'one where I get to meet lots of different people',
 'What was the most number of people you have ever met during a holiday',
 'Hard to keep a count  Maybe twenty five']

In [8]:
# Tokenize the words
tokenizer = get_tokenizer('basic_english')
tokenized_conv = [tokenizer(conv) for conv in preprocessed_lines]

In [9]:
tokenized_conv[2]

['one', 'where', 'i', 'get', 'to', 'meet', 'lots', 'of', 'different', 'people']

In [10]:
tokenized_conv[3]

['what',
 'was',
 'the',
 'most',
 'number',
 'of',
 'people',
 'you',
 'have',
 'ever',
 'met',
 'during',
 'a',
 'holiday']

In [11]:
tokenized_conv[1]

['what', 'is', 'your', 'favorite', 'holiday']

## ```build_vocab_from_iterator```

```build_vocab_from_iterator``` function in the ```torchtext.vocab``` modeule is used to create vocabulary from an iterable of tokenized data. This vocabulary is essential for converting textual data into numerical form.

## Parameters:

### ```tokenized_conv(iterator)```:
This is the main data input to the function. It should be an iterator(like a ```generator``` or a ```list```) that yields sequences to tokens. Each sequences represents a document or an example in your dataset.

### ```min_freq(int,optional):```
This parameter specifics the mimimum frequency a token must have to be inclued in the vocabulary. Tokens that appear fewer than ```min_freq``` times are excluded form the vocabulary. This is useful for removing rare words which might be typos or irrelevant to most analysis.

### ```specials(list of str, optional)```:
This is a list of special tokens that you want to add to the vocabulary. Common special tokens include: A padding token used to equalize the lenghts of sequence.(or for unknown): A token used to represent out-of-vocabulary words during inference, or when a word appears that is not int the traning vocabualry. 

### ```special_first(bool, optional)```:
Determine the ordering of special tokens in the vocabulary. If ```True```, special tokens are added at the beginning of the vocabulary. This can be helpful for certain models where token indices are significant(e.g., models using embadding layers might have specific handling for lower indices.)

In [14]:
features_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_conv,
    min_freq=1,
    specials=['<pad>', '<oov>'],
    special_first=True
)
target_vocab = torchtext.vocab.build_vocab_from_iterator(
    tokenized_conv,
    min_freq=1
)

In [15]:
features_vocab_total_words = len(features_vocab)
target_vocab_toal_words = len(target_vocab)
print("Features Vocab Length:", features_vocab_total_words)
print("Features Vocab Length:", target_vocab_toal_words)

Features Vocab Length: 2749
Features Vocab Length: 2747


In [16]:
# making ngrams from the conversations
def make_ngrams(tokenized_text):
    list_ngrams = []
    for i in range(1, len(tokenized_text)):
        ngram_sequence = tokenized_text[:i+1]
        list_ngrams.append(ngram_sequence)
    return list_ngrams

In [17]:
ngrams_list = []
for tokenized_con in tokenized_conv:
    ngrams_list.extend(make_ngrams(tokenized_con))

In [19]:
ngrams_list[1:2]

[['what', 'is', 'your']]

In [5]:
t2 = []
for i in t:
    t2.extend(i)
print(t2)

TypeError: 'int' object is not iterable